In [1]:
import torch
from vision.ssd.vgg_ssd import create_vgg_ssd, create_vgg_ssd_predictor
from vision.ssd.mobilenetv1_ssd import create_mobilenetv1_ssd, create_mobilenetv1_ssd_predictor
from vision.ssd.mobilenetv1_ssd_lite import create_mobilenetv1_ssd_lite, create_mobilenetv1_ssd_lite_predictor
from vision.ssd.squeezenet_ssd_lite import create_squeezenet_ssd_lite, create_squeezenet_ssd_lite_predictor
from vision.datasets.voc_dataset import VOCDataset
from vision.datasets.open_images import OpenImagesDataset
from vision.utils import box_utils, measurements
from vision.utils.misc import str2bool, Timer
from vision.ssd.data_preprocessing import TrainAugmentation, TestTransform
from vision.ssd.config import vgg_ssd_config
from vision.ssd.ssd import MatchPrior

import argparse
import pathlib
import numpy as np
import logging
import sys

from road_datasets import RoadDamageDataset


In [2]:
DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

def group_annotation_by_class(dataset):
    true_case_stat = {}
    all_gt_boxes = {}
    all_difficult_cases = {}
    for i in range(len(dataset)):
        image_id, annotation = dataset.get_annotation(i)
        gt_boxes, classes, is_difficult = annotation
        gt_boxes = torch.from_numpy(gt_boxes)
        for i, difficult in enumerate(is_difficult):
            class_index = int(classes[i])
            gt_box = gt_boxes[i]
            if not difficult:
                true_case_stat[class_index] = true_case_stat.get(class_index, 0) + 1

            if class_index not in all_gt_boxes:
                all_gt_boxes[class_index] = {}
            if image_id not in all_gt_boxes[class_index]:
                all_gt_boxes[class_index][image_id] = []
            all_gt_boxes[class_index][image_id].append(gt_box)
            if class_index not in all_difficult_cases:
                all_difficult_cases[class_index]={}
            if image_id not in all_difficult_cases[class_index]:
                all_difficult_cases[class_index][image_id] = []
            all_difficult_cases[class_index][image_id].append(difficult)

    for class_index in all_gt_boxes:
        for image_id in all_gt_boxes[class_index]:
            all_gt_boxes[class_index][image_id] = torch.stack(all_gt_boxes[class_index][image_id])
    for class_index in all_difficult_cases:
        for image_id in all_difficult_cases[class_index]:
            all_gt_boxes[class_index][image_id] = torch.tensor(all_gt_boxes[class_index][image_id])
    return true_case_stat, all_gt_boxes, all_difficult_cases


def compute_average_precision_per_class(num_true_cases, gt_boxes, difficult_cases,
                                        prediction_file, iou_threshold, use_2007_metric):
    with open(prediction_file) as f:
        image_ids = []
        boxes = []
        scores = []
        for line in f:
            t = line.rstrip().split(" ")
            image_ids.append(t[0])
            scores.append(float(t[1]))
            box = torch.tensor([float(v) for v in t[2:]]).unsqueeze(0)
            box -= 1.0  # convert to python format where indexes start from 0
            boxes.append(box)
        scores = np.array(scores)
        sorted_indexes = np.argsort(-scores)
        boxes = [boxes[i] for i in sorted_indexes]
        image_ids = [image_ids[i] for i in sorted_indexes]
        true_positive = np.zeros(len(image_ids))
        false_positive = np.zeros(len(image_ids))
        matched = set()
        for i, image_id in enumerate(image_ids):
            box = boxes[i]
            if image_id not in gt_boxes:
                false_positive[i] = 1
                continue

            gt_box = gt_boxes[image_id]
            ious = box_utils.iou_of(box, gt_box)
            max_iou = torch.max(ious).item()
            max_arg = torch.argmax(ious).item()
            if max_iou > iou_threshold:
                if difficult_cases[image_id][max_arg] == 0:
                    if (image_id, max_arg) not in matched:
                        true_positive[i] = 1
                        matched.add((image_id, max_arg))
                    else:
                        false_positive[i] = 1
            else:
                false_positive[i] = 1
        print(len(image_ids))
    true_positive = true_positive.cumsum()
    false_positive = false_positive.cumsum()
    precision = true_positive / (true_positive + false_positive)
    recall = true_positive / num_true_cases
    if use_2007_metric:
        return measurements.compute_voc2007_average_precision(precision, recall),precision,recall
    else:
        return measurements.compute_average_precision(precision, recall)

In [3]:
eval_path = pathlib.Path('eval_results')
eval_path.mkdir(exist_ok=True)
timer = Timer()
label_file = "/data/RoadDamage/labels.txt"
class_names = [name.strip() for name in open(label_file).readlines()]

create_net = create_vgg_ssd
config = vgg_ssd_config


In [4]:
test_transform = TestTransform(config.image_size, config.image_mean, config.image_std)
target_transform = MatchPrior(config.priors, config.center_variance,
                                  config.size_variance, 0.5)
dataset = RoadDamageDataset('/home/chiu/data/RoadDamage/data/', is_test=True)


true_case_stat, all_gb_boxes, all_difficult_cases = group_annotation_by_class(dataset)
net = create_net(len(class_names), is_test=True)

timer.start("Load Model")
net.load('/data/pytorch-ssd/checkpoint/all/VGG-Epoch-300.pth')

net = net.to(DEVICE)

print(f'It took {timer.end("Load Model")} seconds to load the model.')
      
predictor = create_vgg_ssd_predictor(net, nms_method="hard", device=DEVICE)
      

/data/anaconda/envs/neural/lib/python3.7/site-packages/ipykernel_launcher.py:33: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).


It took 1.4236538410186768 seconds to load the model.


In [5]:
results = []
for i in range(len(dataset)):
    print("process image", i)
    timer.start("Load Image")
    image = dataset.get_image(i)
    print("Load Image: {:4f} seconds.".format(timer.end("Load Image")))
    timer.start("Predict")
    boxes, labels, probs = predictor.predict(image,prob_threshold=0.08)
    print("Prediction: {:4f} seconds.".format(timer.end("Predict")))
    indexes = torch.ones(labels.size(0), 1, dtype=torch.float32) * i
    results.append(torch.cat([
            indexes.reshape(-1, 1),
            labels.reshape(-1, 1).float(),
            probs.reshape(-1, 1),
            boxes + 1.0  # matlab's indexes start from 1
    ], dim=1))
results = torch.cat(results)


process image 0
Load Image: 0.172104 seconds.
Inference time:  0.21523094177246094
Prediction: 0.354722 seconds.
process image 1
Load Image: 0.061348 seconds.
Inference time:  0.04061698913574219
Prediction: 0.051543 seconds.
process image 2
Load Image: 0.034354 seconds.
Inference time:  0.03397035598754883
Prediction: 0.052705 seconds.
process image 3
Load Image: 0.055729 seconds.
Inference time:  0.01956319808959961
Prediction: 0.024326 seconds.
process image 4
Load Image: 0.032220 seconds.
Inference time:  0.019470691680908203
Prediction: 0.029247 seconds.
process image 5
Load Image: 0.027910 seconds.
Inference time:  0.01983475685119629
Prediction: 0.024250 seconds.
process image 6
Load Image: 0.038466 seconds.
Inference time:  0.019400358200073242
Prediction: 0.022765 seconds.
process image 7
Load Image: 0.032126 seconds.
Inference time:  0.01954174041748047
Prediction: 0.025980 seconds.
process image 8
Load Image: 0.030819 seconds.
Inference time:  0.019456863403320312
Prediction

In [6]:
for class_index, class_name in enumerate(class_names):
    if class_index == 0: continue  # ignore background
    prediction_path = eval_path / f"det_test_{class_name}.txt"
    with open(prediction_path, "w") as f:
        sub = results[results[:, 1] == class_index, :]
        for i in range(sub.size(0)):
            prob_box = sub[i, 2:].numpy()
            image_id = dataset.ids[int(sub[i, 0])]
            print(
                image_id + " " + " ".join([str(v) for v in prob_box]),
                file=f
            )
aps = []
print("\n\nAverage Precision Per-class:")
for class_index, class_name in enumerate(class_names):
    if class_index == 0:
        continue
    prediction_path = eval_path / f"det_test_{class_name}.txt"
    ap,precision,recall = compute_average_precision_per_class(
        true_case_stat[class_index],
        all_gb_boxes[class_index],
        all_difficult_cases[class_index],
        prediction_path,
        0.5,
        True
    )
    aps.append(ap)
    average_recall = np.sum(recall)/len(recall)
    average_precision = np.sum(precision)/len(precision)
    

    print(f"{class_name}: {ap},precision:{average_precision},recall:{average_recall}")

print(f"\nAverage Precision Across All Classes:{sum(aps)/len(aps)}")



Average Precision Per-class:
9903
D00: 0.4643238272968101,precision:0.2265181614531623,recall:0.6870065636675754
13046
D01: 0.6186394435148349,precision:0.27648517296420533,recall:0.7697601325624783
1786
D10: 0.1059561354895338,precision:0.11555329129600975,recall:0.33260801494509695
7751
D11: 0.11750584318924875,precision:0.050808548312610544,recall:0.3558230389627145
3273
D20: 0.6747303543606832,precision:0.4737073155492176,recall:0.7070954756052458
1264
D40: 0.18419199058879357,precision:0.10982657677866076,recall:0.2845658041697692
1596
D43: 0.8335535982741562,precision:0.5244529161784652,recall:0.8068426486269876
9841
D44: 0.7172509369230228,precision:0.3487954383330026,recall:0.8216198550935783

Average Precision Across All Classes:0.46451901620463537


In [12]:
np.sum(3>1)

1